## Install pyTigerGraph

In [ ]:
# Setup
!pip install -U pyTigerGraph

## Add Imports and Establish Initial Connection

In [ ]:
# Imports
import json
import pandas as pd

from google.colab import drive
import pyTigerGraph as tg

drive.mount('/gdrive')

# Connection parameters
hostName = ''
userName = ''
password = ''
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print('Connected')

## Create Loading Jobs

Let's take a look at what one of our files looks like so we can write a loading job.

In [ ]:
conn.graphname='TigyoreGraph'
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
conn = tg.TigerGraphConnection(host=hostName, graphname='TigyoreGraph', username=userName, password=password, apiToken=authToken)

print('Connected')

In [ ]:
!ls "/gdrive/My Drive/Tigyore/data/nyc"

In [ ]:
!head -n 2 "/gdrive/My Drive/Tigyore/data/nyc/bus_svc.csv"

This one and the messages file will look like a mess in this display because they contain sentances of text as dummy data which makes them not show up too well in the csv preview. You can just reference the header through for loading jobs.

Here it's important to note that the `$0`, `$1` values line up with the columns of your data.

In [ ]:
results = conn.gsql('''
USE GRAPH TigyoreGraph
SHOW VERTEX BusService
SHOW VERTEX BusStop
SHOW EDGE serve
''')

print(results)

In [ ]:
results_svc = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_bus_svc FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX BusService VALUES($0) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

results_stop = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_bus_stops FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX BusStop VALUES($0, $1, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

results_edge = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_serve FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE serve VALUES($0 BusService, $3 BusStop, $2) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

print(results_svc, results_stop, results_edge)

In [ ]:
results = conn.gsql('''
USE GRAPH TigyoreGraph
SHOW JOB load_bus_svc
SHOW JOB load_bus_stops
SHOW JOB load_serve
''')

print(results)

In [ ]:
# Load the bus stops file with the 'load_bus_stops' job
data_file = '/gdrive/My Drive/Tigyore/data/nyc/bus_svc.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_bus_svc')
print(json.dumps(results, indent=2))

In [ ]:
# Load the bus stops file with the 'load_bus_stops' job
data_file = '/gdrive/My Drive/Tigyore/data/nyc/bus_stops_dedup.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_bus_stops')
print(json.dumps(results, indent=2))

In [ ]:
# Load the bus stops file with the 'load_bus_stops' job
data_file = '/gdrive/My Drive/Tigyore/data/nyc/bus_routes.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_serve')
print(json.dumps(results, indent=2))